In [1]:
import numpy as np
from openopt import NLP
import matplotlib.pyplot as plt
%matplotlib inline

Based on papers and genetic algo.

In [7]:
def structtruss(p1, p2, xnum, ynum):
    pinjoints = []
    segments = []
    # making a dotted structure of entire space
    xspan = np.linspace(p1[0], p2[0], xnum + 1)
    yspan = np.linspace(p1[1], p2[1], ynum + 1)
    for y in yspan:
        for x in xspan:
            pinjoints.append([x, y])  # making a grid of dots
    for j in range(ynum):
        for i in range(xnum):
            inter = j * (xnum + 1)
            p1 = i + inter
            p2 = p1 + 1
            p3 = p1 + xnum + 1
            p4 = p3 + 1
            segments.extend([[p1, p2], [p1, p3], [p1, p4], [p2, p3]])
        segments.append([p2, p4])
    idx = ynum * (xnum + 1) + 1
    for j in range(xnum):
        # create segments between pinjoints.
        segments.append([idx + j - 1, idx + j])
    return np.array(pinjoints), np.array(segments)

def genalgtruss(dots, cons, ym, F, freepj, V0, plotdisp=False):
    n = cons.shape[0]
    m = dots.shape[0]
    vecs = dots[cons[:, 1], :] - dots[cons[:, 0], :]
    l = np.sqrt((vecs ** 2).sum(axis=1))
    e = vecs.T / l
    B = (e[np.newaxis] * e[:, np.newaxis]).T

    def freeobj(x):
        D = ym * x / l #Stiffness = E(the cross section area)/length
        kx = e * D
        KZ = np.zeros((2 * m, 2 * m))
        for i in range(n):
            aux = 2 * cons[i, :]
            idx = np.r_[aux[0]:aux[0] + 2, aux[1]:aux[1] + 2]
            K0 = np.concatenate((np.concatenate(
                (B[i], -B[i]), axis=1), np.concatenate((-B[i], B[i]), axis=1)), axis=0)
            KZ[np.ix_(idx, idx)] = KZ[np.ix_(idx, idx)] + D[i] * K0

        box = freepj .flatten().nonzero()[0]
        mat = KZ[np.ix_(box, box)]
        rhs = F.flatten()[box]
        answer = np.linalg.solve(mat, rhs)
        u = freepj .astype("float").flatten()
        u[box] = answer
        U = u.reshape(m, 2)
        axis = ((U[cons[:, 1], :] - U[cons[:, 0], :]) * kx.T).sum(axis=1)
        stress = axis / x
        loss = (U * F).sum()
        strain = -stress ** 2 / ym * l
        return loss, strain, U, stress